In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128521")
exp = Experiment(workspace=ws, name="quick-starts-ws-128521")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128521
Azure region: southcentralus
Subscription id: 5e9e5b82-0802-460e-849e-48901a64399b
Resource group: aml-quickstarts-128521


In [23]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"
try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_node=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True, max_nodes=4)         
    print("Cluster is ready")                                                             

Existing cluster detected, make use of it!


In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
parameter_sampling = RandomParameterSampling( 
                    {
                        "--C":uniform(0.4,1),
                        "--max_iter":choice(0,25,50,75,100) 
                    }
)

#Specify a policy 
policy= BanditPolicy(evaluation_interval=1,slack_factor=0.2)
 
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#import packages
from azureml.core import ScriptRunConfig
from azureml.core import Environment

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
script='train.py'
est=SKLearn(
    source_directory=script_folder,
    entry_script='train.py',
    compute_target=new_cluster
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                              hyperparameter_sampling=parameter_sampling,
                              policy=policy,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=4,
                              max_concurrent_runs=4
                                     )

In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [46]:
import joblib
# Get your best run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)
# Save the model from the run.
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name = 'HyperDriveModel', model_path = 'outputs/model.joblib')
print(best_run)

{'Regularization Strength:': 0.9593686692387857, 'Max iterations:': 75, 'Accuracy': 0.9072837632776934}
['azureml-logs/55_azureml-execution-tvmps_801036204c379a1acfcf8ffd0ff36e396132335045042555e6391a3a4acc868d_d.txt', 'azureml-logs/65_job_prep-tvmps_801036204c379a1acfcf8ffd0ff36e396132335045042555e6391a3a4acc868d_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_801036204c379a1acfcf8ffd0ff36e396132335045042555e6391a3a4acc868d_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/105_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Run(Experiment: quick-starts-ws-128521,
Id: HD_709b2ee0-0773-4637-8e06-bac4629c87dd_2,
Type: azureml.scriptrun,
Status: Completed)


In [39]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=src)

In [40]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [41]:
#splitting the dataset
from sklearn.model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test=train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=42)
train_data=pd.concat((x_train,y_train),axis=1)

In [42]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data='train_data',
    label_column_name='y',
    n_cross_validations=2,
    featurization='auto',
    max_concurrent_iterations=4,
    max_cores_per_iteration=4)

In [43]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.train.automl.run import AutoMLRun

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on local machine


ValidationException: ValidationException:
	Message: An empty value for argument [X] is provided.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "An empty value for argument [X] is provided.",
        "target": "X",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentBlankOrEmpty"
            }
        }
    }
}